# Boosting
-- Thiago Vieira de Alcantara Silva<br/>
-- 2017719891

In [13]:
import pandas as pd
import numpy as np

RSEED = 42
np.random.seed(RSEED)

### Reading the dataset...

In [14]:
data = pd.read_csv('tic-tac-toe.data', header=None, delimiter=',')
data.head()

,0,1,2,3,4,5,6,7,8,9
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


### Treating the dataset...
Since the target variable is *positive* whenever the player x won the game, we consider both **o** and **b** as in the same class.

In [15]:
data = data.applymap(lambda x: 1 if (x == 'x' or x == 'positive') else 0)
data.describe()

,0,1,2,3,4,5,6,7,8,9
count,958.000000,958.000000,958.000000,958.000000,958.000000,958.000000,958.000000,958.000000,958.000000,958.000000
mean,0.436326,0.394572,0.436326,0.394572,0.478079,0.394572,0.436326,0.394572,0.436326,0.653445
std,0.496188,0.489014,0.496188,0.489014,0.499780,0.489014,0.496188,0.489014,0.496188,0.476121
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
class AdaBoost():
    def __init__(self):
        pass
    
    def train(self, data):
        '''
            This function trains the model on the data received.
        '''
        self.bweights = [1.0/data.shape[0] for i in range(data.shape[0])]
        self.alpha = []
        
        
    def test(self, data):
        '''
            This function runs the model on the data received and returns the
            accuracy.
        '''
        pass
    
    
    def get_best_stump(self):
        '''
            This function gets the stump that minimizes the empirical error,
            and add it to the list of stumps with its alpha value.
        '''
        pass
    
    
    def update_bweights(self):
        pass